# HW4 Trees and Forests
The analysis task, which aims to classify sentiments using bag-of-words features of product reviews, will let you:
* get familiar with hyper-parameters of decision trees and ensemble models;
* understand the relative strength of trees and ensemble methods;
* gain experience of training these models with `sklearn`.

In [57]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os
import sys
import time
import sklearn.tree
import sklearn.linear_model
import sklearn.metrics
import sklearn.ensemble

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
# Plotting utils
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')

In [59]:
# From the HW4 starter code
from pretty_print_sklearn_tree import pretty_print_sklearn_tree

# Load all data from train/valid/test

We consider the text sentiment classification task as in Project A where each example is one plain-text online review collect from amazon.com for a consumer product (either a book, a movie, an electronics item, or a kitchen item). Dataset credit: M. Dredze, J. Blitzer, and Fernando Pereira. <https://www.cs.jhu.edu/~mdredze/datasets/sentiment/>


Here's an example book review that is a positive sentiment:

> This all-Spanish handbook for parents with new babies will prove essential for any concerned about a child's health. 

Here's one with a negative sentiment

> It completely sucked. Very long with nothing to say. Author was proud of his own knowledge of the SCA and Wiccans, and the publishers thought that obviated the need for plot or character development.



We have preprocessed this data for you to the *bag-of-words* representation, using a fixed vocabulary of the 7729 most common words provided by the original dataset creators (with some slight modifications by us). The vocabulary includes some *bigrams* (e.g. "waste_of") in addition to single words.  In this task, we use binary features indicating whether words are present (1) or absent (0) in the text. 

Each data subset is provided as a CSV file. In the header, each column name is the word corresponding to the feature: the feature value indicates whether the corresponding word is present (1) or absent (0) in the review. Later, you will show features that are important for the classification task. It means that the words corresponding to these features are informative about class labels.

The label is an overall binary rating of the user's feelings about the product (either 'positive' or 'negative'). Our goal is to build a model that can predict the sentiment (positive or negative) from the text alone.

We have provided the following:

* a training set of 6346 instances
* a validation set of 792 instances
* a test set of 793 instances


In [24]:
# TODO fix to path on your local system
DATA_DIR = os.path.join("data_product_reviews/")

### Load training

In [60]:
x_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'x_train.csv.zip'))
y_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'y_train.csv'))
x_tr_NF = np.minimum(x_tr_df.values, 1.0).copy()
y_tr_N = y_tr_df.values[:,0].copy()

print("Training data")
print("x_tr_NF.shape: %s" % str(x_tr_NF.shape))
print("y_tr_N.shape : %s" % str(y_tr_N.shape))
print("mean(y_tr_N) : %.3f" % np.mean(y_tr_N))

Training data
x_tr_NF.shape: (6346, 7729)
y_tr_N.shape : (6346,)
mean(y_tr_N) : 0.500


### Load validation set

In [61]:
x_va_df = pd.read_csv(os.path.join(DATA_DIR, 'x_valid.csv.zip'))
y_va_df = pd.read_csv(os.path.join(DATA_DIR, 'y_valid.csv'))

x_va_TF = np.minimum(x_va_df.values, 1.0).copy()
y_va_T = y_va_df.values[:,0].copy()

print("Validation data")
print("x_va_TF.shape: %s" % str(x_va_TF.shape))
print("y_va_T.shape : %s" % str(y_va_T.shape))
print("mean(y_va_T) : %.3f" % np.mean(y_va_T))

Validation data
x_va_TF.shape: (792, 7729)
y_va_T.shape : (792,)
mean(y_va_T) : 0.490


### Load test set 

In [62]:
x_te_df = pd.read_csv(os.path.join(DATA_DIR, 'x_test.csv.zip'))
y_te_df = pd.read_csv(os.path.join(DATA_DIR, 'y_test.csv'))

x_te_TF = np.minimum(x_te_df.values, 1.0).copy()
y_te_T = y_te_df.values[:,0].copy()

print("Heldout Test data")
print("x_te_TF.shape: %s" % str(x_te_TF.shape))
print("y_te_T.shape : %s" % str(y_te_T.shape))
print("mean(y_te_T) : %.3f" % np.mean(y_te_T))

Heldout Test data
x_te_TF.shape: (793, 7729)
y_te_T.shape : (793,)
mean(y_te_T) : 0.515


### Show column names
Each column name is a word. A feature value (1 or 0) indicates whether the word is present or absent in the review. 

In [63]:
vocab_list = x_tr_df.columns.tolist()

for word in vocab_list[:8]:
    print(word)
print("...")
for word in vocab_list[-8:]:
    print(word)

good
great
time
book
don't
work
i_have
read
...
never_get
i'd_like
loves_it
an_author
nomin
could_give
bad_but
gap


### Pack training and validation sets into big arrays (so we can use sklearn's hyperparameter search tools)

In [64]:
xall_LF = np.vstack([x_tr_NF, x_va_TF])
yall_L = np.hstack([y_tr_N, y_va_T])

# Create splitter object using Predefined Split
# Will be used later by all hyperparameter sear
## <a id="problem-1">Problem 1</a>: Decision Trees for Review Classification 

valid_indicators_L = np.hstack([
    -1 * np.ones(y_tr_N.size), # -1 means never include this example in any test split
    0  * np.ones(y_va_T.size), #  0 means include in the first test split (we count starting at 0 in python)
    ])
my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)

# Problem 1: Train Decision Trees for Sentiment Classification

## Implementation A.1 : Train a Simple Tree

Train a `DecisionTreeClassifier` with `criterion='gini'`, `max_depth=3`, `min_samples_leaf=1`, `min_samples_split=2` and `random_state=101`.

You'll use this simple tree again later (to make Table 3).

**TODO**: Fit the tree  on the training set in the next coding cell



In [65]:
simple_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini', 
    max_depth=3, min_samples_split=2, min_samples_leaf=1, 
    random_state=101)

In [66]:
# TODO
simple_tree.fit(x_te_df, y_te_df)

DecisionTreeClassifier(max_depth=3, random_state=101)


## Report 1a: Figure 1 in the report

Show the ASCII-text representation of your trained decision tree from 1A, by calling the helper function `pretty_print_sklearn_tree` found in the starter code.
(Remember when you are reading the printed statements that "Y" means the above decision question evaluated to "yes" and "N" means "no".)


In [67]:
# TODO 
# call pretty_print_sklearn_tree on simple_tree, providing feature_names=vocab_list
pretty_print_sklearn_tree(simple_tree, vocab_list)

The binary tree structure has 15 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    8 are leaves
The decision tree:  (Note: Y = 'yes' to above question; N = 'no')
Decision: X['great'] <= 0.50?
  Y Decision: X['waste'] <= 0.50?
    Y Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.504 (496 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.130 (46 total training examples)
    N Decision: X['so-cal'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.000 (36 total training examples)
      N Leaf: p(y=1 | this leaf) = 1.000 (1 total training examples)
  N Decision: X['bad'] <= 0.50?
    Y Decision: X['terribl'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.777 (188 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.125 (8 total training examples)
    N Decision: X['bought'] <= 0.50?
      Y Leaf:

## Implementation A.2: Model Selection

Perform a grid search for the hyperparameters of your `DecisionTree` over the following settings:

  * `max_depth` in [2, 8, 32, 128]
  * `min_samples_leaf` in [1, 3, 9]
  * `random_state` in [101]

Be sure to use [sklearn.model_selection.GridSearchCV]().

Additional requirements (keep all other settings at default values)

* Set `scoring='accuracy'`, since our target metric is accuracy
* Set `cv=my_splitter` (as in starter code), so you can use the predefined split we defined earlier.
* Set `return_train_score=True`, since we want training set scores as well as test set scores
* Set `refit=False`, because we only want fits on `x_tr_NF` (for later with table 3)

You'll use this "best" tree again later (to make Table 3).


In [68]:
# Construct the default predictor
# Any hyperparameters here may be overridden by the hyperparameter grid

base_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini',
    min_samples_split=2, min_samples_leaf=1)

In [69]:
tree_hyperparameter_grid_by_name = dict(
    max_depth=[2, 8, 32, 128],
    min_samples_leaf=[1, 3, 9],
    random_state = [101],
    )

**TODO:** Build the Grid Search in the next coding cell

In [77]:
import sklearn.model_selection


tree_grid_searcher = sklearn.model_selection.GridSearchCV(estimator=base_tree, param_grid=tree_hyperparameter_grid_by_name, scoring='roc_auc')

**TODO**: Do the search!


In [78]:
start_time_sec = time.time()
tree_grid_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

**Move the results of grid search into a nice pandas data frame.**

In [79]:
if hasattr(tree_grid_searcher, 'cv_results_'):
    # Will be here if you called fit on tree_grid_searcher succesfully
    tree_search_results_df = pd.DataFrame(tree_grid_searcher.cv_results_).copy()
    print("Grid search of %3d configurations done after %6.1f sec" % (
        tree_search_results_df.shape[0], elapsed_time_sec))

Grid search of  12 configurations done after 49971.2 sec


**Display search results.** This block will make a pretty printed table of the results of your grid search

In [82]:
if hasattr(tree_grid_searcher, 'cv_results_'):
    pd.set_option('display.precision', 4)
    tree_keys = ['param_max_depth', 'param_min_samples_leaf']
    tree_search_results_df.sort_values(tree_keys, inplace=True)
    tree_search_results_df[tree_keys + ['mean_test_score', 'rank_test_score', 'mean_fit_time']]

In [83]:
print("Printing a dict of the best hyperparameters")
try:
    print(tree_grid_searcher.best_params_)
except AttributeError:
    print("TODO")

Printing a dict of the best hyperparameters
{'max_depth': 128, 'min_samples_leaf': 9, 'random_state': 101}


## Implementation A.3: Build the Best Decision Tree

**TODO Build the Best Tree on the training set** in the next coding cell. This is necessary so you have the specific best performing tree in your workspace. Although you fit many trees in the search, they were not stored, so we need to recreate the best one. Hint: Just feed the best hyperparameters as keyword args to construct the tree and fit it to the training data.

In [84]:
best_tree = base_tree.set_params(max_depth = 128, min_samples_leaf = 9, random_state = 101) # TODO call set_params using the best_params_ found by your searcher
best_tree.fit(x_tr_NF, y_tr_N)

DecisionTreeClassifier(max_depth=128, min_samples_leaf=9, random_state=101)


## Report 1b: short answers to the following two questions

* What are the `max_depth` and `min_samples_leaf` of the best tree?
* How should we change `min_samples_leaf` if we want to reduce overfitting? Please explain.



## Report 1c: plot the best decision tree

Please include the plot of the best decision tree in the plot.

In [130]:
c_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini',
    min_samples_split=2, min_samples_leaf=1)

c_tree_hyperparameter_grid_by_name = dict(
    max_depth=[3],
    min_samples_leaf=[1, 3, 9, 27],
    min_samples_split = [0.01, 0.04, 0.08, 0.16],
    random_state = [101],
    )

import sklearn.model_selection

c_tree_grid_searcher = sklearn.model_selection.GridSearchCV(estimator=c_tree, param_grid=c_tree_hyperparameter_grid_by_name, scoring='roc_auc')

start_time_sec = time.time()
c_tree_grid_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

if hasattr(tree_grid_searcher, 'cv_results_'):
    # Will be here if you called fit on tree_grid_searcher succesfully
    c_tree_search_results_df = pd.DataFrame(c_tree_grid_searcher.cv_results_).copy()
    print("Grid search of %3d configurations done after %6.1f sec" % (
        c_tree_search_results_df.shape[0], elapsed_time_sec))
    
print("Printing a dict of the best hyperparameters")
try:
    print(c_tree_grid_searcher.best_params_)
except AttributeError:
    print("TODO")


Grid search of  16 configurations done after  148.8 sec
Printing a dict of the best hyperparameters
{'max_depth': 3, 'min_samples_leaf': 27, 'min_samples_split': 0.04, 'random_state': 101}


In [131]:
best_c_tree = c_tree.set_params(max_depth = 3, min_samples_leaf = 27, min_samples_split = 0.04, random_state = 101) # TODO call set_params using the best_params_ found by your searcher
best_c_tree.fit(x_tr_NF, y_tr_N)
    
if hasattr(tree_grid_searcher, 'cv_results_'):
    pd.set_option('display.precision', 4)
    c_tree_keys = ['param_max_depth', 'param_min_samples_leaf']
    c_tree_search_results_df.sort_values(c_tree_keys, inplace=True)
    c_tree_search_results_df[c_tree_keys + ['mean_test_score', 'rank_test_score', 'mean_fit_time']]
pretty_print_sklearn_tree(best_c_tree, feature_names=vocab_list)

The binary tree structure has 13 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    1 are leaves
- depth   3 has    6 nodes, of which    6 are leaves
The decision tree:  (Note: Y = 'yes' to above question; N = 'no')
Decision: X['great'] <= 0.50?
  Y Decision: X['excel'] <= 0.50?
    Y Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.430 (4041 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.114 (368 total training examples)
    N Decision: X['it_was'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.905 (263 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.643 (28 total training examples)
  N Decision: X['return'] <= 0.50?
    Y Decision: X['bad'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.745 (1413 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.415 (142 total training examples)
    N Leaf: p(y=1 | this leaf) = 0.275 (91 t

## Report 1d: a short Answer 1b in Report

Reflect on the two trees plotted. Remember that we are predicting overall sentiment about a product based on observed words in a written review.

* Is there any internal node that has two child leaf nodes corresponding to the same sentiment class? If so, why would having two children predict the same class make sense? (Hint: do the two child leaves have the same output in the printout? What does this have to do with gini impurity?)


# Problem 2: Training a Random Forest for Sentiment Classification

**An example tabl** 

|**Important Words**|**Unimportant Words**|
|:-:|:-:|
|I1 |  U1  |
|I2 |  U2  |
|I3 |  U3  |
|I4 |  U4  |
|I5 |  U5  |
|I6 |  U6  |
|I7 |  U7  |
|I8 |  U8  |
|I9 |  U9  |
|I0 |  U0  |

## Implementation B.1: Find good hyperparameters for a Random Forest via grid search

Follow the instructions and use what you learn in Problem 1 to finish this step. This block might take 2-10 minutes. (Takes about 2 min on staff Macbook laptops.) If yours runs significantly longer, try this out on Google Colab instead.

In [86]:
base_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=16,
    min_samples_split=2,
    min_samples_leaf=1)


In [87]:
forest_hyperparameter_grid_by_name = dict(
    max_features=[3, 10, 33, 100, 333],
    max_depth=[16, 32],
    min_samples_leaf=[1],
    n_estimators=[100],
    random_state=[101],
    )

In [88]:
# TODO construct a GridSearchCV object like you did above.

forest_searcher = sklearn.model_selection.GridSearchCV(estimator=base_forest, param_grid=forest_hyperparameter_grid_by_name) 

**TODO**: Do the search!

In [89]:
#TODO
start_time_sec = time.time()
forest_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

**TODO**: Build dataframe of results. You can reuse the code from Problem 1

In [92]:
#TODO
if hasattr(forest_searcher, 'cv_results_'):
    # Will be here if you called fit on forest_searcher succesfully
    forest_search_results_df = pd.DataFrame(forest_searcher.cv_results_).copy()
    print("Grid search of %3d configurations done after %6.1f sec" % (
        forest_search_results_df.shape[0], elapsed_time_sec))


Grid search of  10 configurations done after  416.9 sec


**TODO**: Display search results

In [124]:
#TODO
if hasattr(forest_searcher, 'cv_results_'):
    pd.set_option('display.precision', 4)
    forest_keys = ['param_max_features', 'param_max_depth', 'param_min_samples_leaf', 'param_n_estimators']
    forest_search_results_df.sort_values(forest_keys, inplace=True)
    forest_search_results_df[forest_keys + ['mean_test_score', 'rank_test_score', 'mean_fit_time']]

print("Printing a dict of the best hyperparameters")
try:
    print(forest_searcher.best_params_)
except AttributeError:
    print("TODO")

Printing a dict of the best hyperparameters
{'max_depth': 32, 'max_features': 33, 'min_samples_leaf': 1, 'n_estimators': 100, 'random_state': 101}


## Implementation B.2: Build the best random forest using the best hyperparameters found in B.1 

This is necessary so you have the specific best performing forest in your workspace.

Train *only* on training set (do not merge train and valid)


In [94]:
#TODO
best_forest = base_forest.set_params(max_depth = 32, max_features = 33, min_samples_leaf = 1, n_estimators = 100, random_state = 101) # TODO call set_params using the best_params_ found by your searcher
best_forest.fit(x_tr_NF, y_tr_N)


RandomForestClassifier(max_depth=32, max_features=33, random_state=101)

## Implementation B.3: Compute feature importances from the trained random forest


Please check <https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html>. We did not talk about feature importances in the class, but this is an important concept for you to learn. It is easy to compute feature importances with `sklearn`. 

In [110]:
# TODO Compute feature importances here. 
import time

import numpy as np

start_time = time.time()
importances = best_forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in best_forest.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

# Sort the feature importances and get the indices
indices = np.argsort(importances)



# Get the top 10 most important and least important features
top_indices = indices[-10:]
bottom_indices = indices[:10]

# Create a DataFrame to store the results
data = {
    'Top Words': [vocab_list[i] for i in top_indices],
    'Bottom Words': [vocab_list[i] for i in bottom_indices],
}

# Create the DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

threshold = 0.00001
count_low_importance = sum(imp < threshold for imp in importances)
print("Words with importance below threshold: ", count_low_importance)

Elapsed time to compute the importances: 0.017 seconds
    Top Words   Bottom Words
0     easy_to      first_few
1       worst    years_after
2        poor      always_be
3        love       me_about
4       excel          domin
5       great  several_other
6  disappoint      the_night
7         bad        you_had
8      return   toaster_oven
9       waste       right_to
Words with importance below threshold:  791


## Report 2a: Show important and unimportant words in the classification task 

Please read this page <https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html> to understand feature importances. You need to create a table and show 10 important words and 10 unimportant words. You can show a few more if you find the results interesting. 

An example table is shown below. The table has 2 panels, with the left one displaying 10 vocabulary words with *highest* feature importance, and the right one displaying 10 randomly chosen terms that have *close-to-zero* feature importance (any feature with importance less than 0.00001 is eligible). For the latter one, **Be sure to indicate how many 'eligible' features these 10 were chosen from.**

*Hint*: useful functions: `numpy.argsort`, `numpy.flatnonzero`, `numpy.abs`. There is a sample output in the Jupyter notebook.

**No caption is needed**. But we do encourage you to reflect for your own learning: do the top-ranked words seem important to your own human judgment? 



**An example table** 

|**Important Words**|**Unimportant Words**|
|:-:|:-:|
|I1 |  U1  |
|I2 |  U2  |
|I3 |  U3  |
|I4 |  U4  |
|I5 |  U5  |
|I6 |  U6  |
|I7 |  U7  |
|I8 |  U8  |
|I9 |  U9  |
|I10 |  U10  |

## Report 2b: A short answer to the following questions

* What is the best hyper-parameter combination from your search?
* What is the maximum value `max_features` could take for this dataset?
* Why do we want to tune `max_features` instead of setting it to its max possible value?

## Report 2c: A short answer to the following questions

When you fit random forests, you can adjust `n_estimators` to set the number of trees in your ensemble.

* What is the primary tradeoff this hyperparameter controls? 
* Can you overfit by setting this too large? Why or why not? You may need to experiment with this hyper-parameter to answer the question.


# Problem 3: Train a GBDT for Sentiment Classification


We'll now try another kind of *ensemble* method: Gradient Boosting to improve the performance for Decision Trees on our classification problem.


## Implementation C.1: Best GBDT via grid search

This step is similar to previous model selection. This block might take several minutes. If yours runs significantly longer, try this out on Google Colab instead.

In [117]:
base_gbdt = sklearn.ensemble.GradientBoostingClassifier(
    n_estimators=100,
    criterion='friedman_mse',
    max_depth=16,
    min_samples_split=2,
    min_samples_leaf=1)

In [118]:
# NOTE: this is the recommended search range. You can choose to make some adjustments so 
# that the best model is not on the "boundary" of the search range. 

gbdt_hyperparameter_grid_by_name = dict(
    max_features=[3],
    max_depth=[4, 8, 16, 32],
    learning_rate=[0.1, 0.01, 0.001],
    n_estimators=[100, 200, 300],
    random_state=[101],
    )

In [119]:
# TODO construct a GridSearchCV object like you did above.

gbdt_searcher = sklearn.model_selection.GridSearchCV(estimator=base_gbdt, param_grid=gbdt_hyperparameter_grid_by_name) 

**TODO**: do the search!

In [120]:
#TODO
start_time_sec = time.time()
gbdt_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

**TODO**: Build dataframe of results

In [121]:
#TODO
if hasattr(gbdt_searcher, 'cv_results_'):
    # Will be here if you called fit on forest_searcher succesfully
    gbdt_search_results_df = pd.DataFrame(gbdt_searcher.cv_results_).copy()
    print("Grid search of %3d configurations done after %6.1f sec" % (
        gbdt_search_results_df.shape[0], elapsed_time_sec))


Grid search of  36 configurations done after  390.3 sec


**TODO**: Display search results

In [125]:
#TODO
if hasattr(gbdt_searcher, 'cv_results_'):
    pd.set_option('display.precision', 4)
    gbdt_keys = ['param_max_features','param_max_depth','param_learning_rate','param_n_estimators','param_random_state']
    gbdt_search_results_df.sort_values(gbdt_keys, inplace=True)
    gbdt_search_results_df[gbdt_keys + ['mean_test_score', 'rank_test_score', 'mean_fit_time']]

print("Printing a dict of the best hyperparameters")
try:
    print(gbdt_searcher.best_params_)
except AttributeError:
    print("TODO")

Printing a dict of the best hyperparameters
{'learning_rate': 0.01, 'max_depth': 32, 'max_features': 3, 'n_estimators': 300, 'random_state': 101}


## Implementation C.2: Build the best GBDT using the best hyperparameters found in C.1

This is necessary so you have the specific best performing forest in your workspace.

Train *only* on training set (do not merge train and valid)


In [126]:
#TODO
best_gbdt = base_gbdt.set_params(learning_rate = 0.01, max_depth = 32, max_features = 3, n_estimators = 300, random_state = 101) 
best_gbdt.fit(x_tr_NF, y_tr_N)

GradientBoostingClassifier(learning_rate=0.01, max_depth=32, max_features=3,
                           n_estimators=300, random_state=101)

## Report 3a: Short answers to the following questions

* What is the best hyper-parameter combination from your search?
* Is the hyperparameter `n_estimators` related to overfitting/underfitting behaviors? Please explain.
* If we use a shallow tree, which could not fit the data well (high bias). Can GBDT still work? Please explain your answers. 

## Report 3b: A short answer to the following question.

We are solving a classification problem, but the setting `criterion='friedman_mse'` suggests that some kind of squared error is minimized. Is that weird? Can you resolve this question with a short explanation? 

## Report 3c: Report feature importances

Please create a table and show the top 10 and the bottom 10 important words. You can show a few more if you find the results interesting. The table should have the same format as the one in Problem 2. 


In [127]:
# TODO Compute feature importances here. 
import time

import numpy as np

start_time = time.time()
importances = best_gbdt.feature_importances_
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

# Sort the feature importances and get the indices
indices = np.argsort(importances)



# Get the top 10 most important and least important features
top_indices = indices[-10:]
bottom_indices = indices[:10]

# Create a DataFrame to store the results
data = {
    'Top Words': [vocab_list[i] for i in top_indices],
    'Bottom Words': [vocab_list[i] for i in bottom_indices],
}

# Create the DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

threshold = 0.00001
count_low_importance = sum(imp < threshold for imp in importances)
print("Words with importance below threshold: ", count_low_importance)

Elapsed time to compute the importances: 0.016 seconds
    Top Words Bottom Words
0        poor          con
1     a_great    ending_is
2        bore        domin
3    the_best      in_just
4         bad    one_point
5     easy_to       sunday
6  disappoint         host
7       waste     the_hero
8       excel   i_strongly
9       great      hold_it
Words with importance below threshold:  69


# Problem 4: Compare Different Models


## Report 4a: Compare accuracies of different models

Please report **accuracy values** of the three best models on the train, valid, and test sets, to 3 digits of precision in a table.

The table should include 3 rows, one for each model pipeline:

* Best decision tree (from 1B)
* Best random forest (from 2C)
* Best GBDT (from 3C)

It should have 5 columns:

* col 1 to indicate the *number of trees*
* col 2 to indicate the `max_depth` used (or found via search)
* col 3-5 report *balanced accuracy* on each dataset (train/valid/test)

Include a brief caption summarizing your conclusions: 

* Which method (which row) does best on the test set?
* Does your table's relative ranking of "best" tree, "best" forest, and "best" GBDT agree with course concepts? Why or why not?

**Sample Output** (Feel free to print all values and organize them by hand)

|**method**|**max depth**|**num trees**|**train BAcc**|**valid BAcc**|**test BAcc**|
|:-|:-:|:-:|:-:|:-:|:-:|
|best Tree	|1 | 1 | 0.123	|0.456	|0.890|
|best RandomForest	|1 | 1 | 0.123	|0.456	|0.890|
|best GBDT	|1 | 1 | 0.123	|0.456	|0.890|

In [137]:
import pandas as pd
from sklearn.metrics import balanced_accuracy_score

# Assuming you have predictions for training, validation, and test sets for each model
# Replace these with your actual predictions
# Make sure that your predictions are in the format of true labels and predicted labels

# Calculate BAcc scores for training, validation, and test sets for each model
best_tree_train_bacc = balanced_accuracy_score(y_tr_df, best_tree.predict(x_tr_df))
best_tree_val_bacc = balanced_accuracy_score(y_va_df, best_tree.predict(x_va_df))
best_tree_test_bacc = balanced_accuracy_score(y_te_df, best_tree.predict(x_te_df))

best_forest_train_bacc = balanced_accuracy_score(y_tr_df, best_forest.predict(x_tr_df))
best_forest_val_bacc = balanced_accuracy_score(y_va_df, best_forest.predict(x_va_df))
best_forest_test_bacc = balanced_accuracy_score(y_te_df, best_forest.predict(x_te_df))

best_gbdt_train_bacc = balanced_accuracy_score(y_tr_df, best_gbdt.predict(x_tr_df))
best_gbdt_val_bacc = balanced_accuracy_score(y_va_df, best_gbdt.predict(x_va_df))
best_gbdt_test_bacc = balanced_accuracy_score(y_te_df, best_gbdt.predict(x_te_df))

# Create a DataFrame to store the results
data = {
    'Method': ['Decision Tree', 'Random Forest', 'Gradient Boosted Trees'],
    'Max Depth': [best_tree.max_depth, best_forest.max_depth, best_gbdt.max_depth],
    'Num Trees': [1, best_forest.n_estimators, best_gbdt.n_estimators],
    'Train BAcc': [best_tree_train_bacc, best_forest_train_bacc, best_gbdt_train_bacc],
    'Validation BAcc': [best_tree_val_bacc, best_forest_val_bacc, best_gbdt_val_bacc],
    'Test BAcc': [best_tree_test_bacc, best_forest_test_bacc, best_gbdt_test_bacc]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


c:\Users\carey\anaconda3\envs\cs135_env\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\carey\anaconda3\envs\cs135_env\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\carey\anaconda3\envs\cs135_env\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\carey\anaconda3\envs\cs135_env\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\carey\anaconda3\envs\cs135_env\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\carey\anaconda3\envs\cs135_env\lib\site-p

                   Method  Max Depth  Num Trees  Train BAcc  Validation BAcc  \
0           Decision Tree        128          1      0.8308           0.7296   
1           Random Forest         32          1      0.9690           0.8373   
2  Gradient Boosted Trees         32        300      0.9943           0.8487   

   Test BAcc  
0     0.7127  
1     0.8342  
2     0.8522  
